In [ ]:
#Source: https://stackoverflow.com/questions/58812642/unable-to-scrape-the-conversation-among-debaters-in-order-to-put-them-in-a-dicti
import requests, re
from bs4 import BeautifulSoup as bs
from IPython.display import HTML
import pprint
from bs4 import BeautifulSoup


debates = requests.get('http://www.presidency.ucsb.edu/debates.php')
soup = BeautifulSoup(debates.content, 'html.parser')
data = []
links = []

def remove_duplicates(l):
    for item in l:
        match = re.search("(?P<url>https?://[^\s]+)", item)
        if match is not None:
            links.append((match.group("url")))


for link in soup.select('#block-system-main a', href=True):
    data.append(str(link.get('href')))
flag = True
remove_duplicates(data)

for url in links:
    print(url)


In [ ]:
#Source: https://stackoverflow.com/questions/58812642/unable-to-scrape-the-conversation-among-debaters-in-order-to-put-them-in-a-dicti
results = {}
p = re.compile(r'\b(\w+)\b\s+\(|\b(\w+)\b,')

with requests.Session() as s:
    for number, link in enumerate(links):
        r = s.get(link)
        soup = bs(r.content,'lxml')
        try:
            participants_tag = soup.select_one('p:has(strong:contains("PARTICIPANTS:"))')
            if participants_tag.select_one('strong'):
                participants_tag.strong.decompose()
            speaker_dict = {i[0].upper() + ':' if i[0] else i[1].upper() + ':': [] for string in participants_tag.stripped_strings for i in p.findall(string)}
            print(speaker_dict)
            moderator_data = [string for string in soup.select_one('p:has(strong:contains("MODERATOR:","MODERATORS:"))').stripped_strings][1:]
            moderators = [i[0].upper() + ':' if i[0] else i[1].upper() + ':' for string in moderator_data for i in p.findall(string)]
            store_paragraph = False

            for paragraph in soup.select('.field-docs-content p:not(p:contains("PARTICIPANTS:","MODERATOR:"))')[1:]:
                string_to_compare = paragraph.text.split(':')[0] + ':'
                string_to_compare = string_to_compare.upper()
                if string_to_compare in moderators:
                    store_paragraph = False
                elif string_to_compare in speaker_dict:
                    speaker = string_to_compare
                    store_paragraph = True
                if store_paragraph:
                    speaker_dict[speaker].append(paragraph.text)
            results[number] = speaker_dict
        except: 
            pass            

In [ ]:
import csv

with open('debates_v3.csv', 'w') as csv_file:  
    writer = csv.writer(csv_file)
    for key, value in results.items():
       writer.writerow([key, value])

In [ ]:
import json

with open('results.json', 'w') as fp:
    json.dump(results, fp)